In [2]:
imagefile= r'C:\Users\roumba\Documents\Software\deep-events\Videos\Video_1\Brightfield\220915_cos7_mitostaygold1_w1Brightfield.ome.tif'
csvfile = r'C:\Users\roumba\Documents\Software\deep-events\Videos\Video_1\Brightfield\labels1b.csv'

from PIL import Image
import pandas as pd
import numpy as np

img = Image.open(imagefile)
#tiffarray = np.array(img)

data = pd.read_csv(csvfile)
#data.columns


498

In [ ]:
for excelindex in range len(data):
    

In [ ]:
frame1=int(datacrop['axis-0'].min())
frame2=int(datacrop['axis-0'].max())
ymean = datacrop['axis-1'].mean()
xmean = datacrop['axis-2'].mean()
ycrop1=ymean+128
ycrop2=ymean-128
xcrop1=xmean+128
xcrop2=xmean-128

In [ ]:
dataar=np.zeros((frame2-frame1+1, 256, 256))

for frameindex, framenumber in enumerate(range (frame1, frame2)):
    img.seek(framenumber-1) #starts from 0 I think?
    box = (xcrop2, ycrop2, xcrop1, ycrop1) #choose dimensions of box
    imcrop= img.crop(box)
    
    dataar[frameindex, :, :] = np.array(imcrop)
    
print(dataar.shape)
